In [ ]:
from transformers import pipeline, AutoModelForTokenClassification, AutoTokenizer
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from simpletransformers.ner import NERModel, NERArgs

In [ ]:
# bert model -- more personal data cleaning
df=pd.read_csv("Train_Tagged_Titles.tsv.gz", sep="\t", on_bad_lines='skip')
agg_d = {'Record Number': 'first', 'Title': 'first',
         'Token': lambda x: ' '.join(x.dropna()),
          'Tag': 'first'}

df = df.groupby(df.Tag.notnull().cumsum().rename(None)).agg(agg_d)
df['Token'] = df.Token.str.lower()
df = df.drop_duplicates()
df = df[df['Tag'].notnull()]

In [ ]:
df.describe()

In [ ]:
X = df[['Record Number', 'Token']]
Y = df[['Tag']]

x_train, x_test, y_train, y_test = train_test_split(X,Y, test_size =0.2)

In [ ]:
label = df["Tag"].unique().tolist()
label


In [ ]:
args = NERArgs()
args.num_train_epochs = 1
args.learning_rate = 1e-4
args.overwrite_output_dir =True
args.train_batch_size = 32
args.eval_batch_size = 32


In [ ]:
train = pd.DataFrame({"sentence_id":x_train["Record Number"], "words":x_train["Token"], "labels":y_train["Tag"]})
test = pd.DataFrame({"sentence_id":x_test["Record Number"], "words":x_test["Token"], "labels":y_train["Tag"]})

In [ ]:
model = NERModel('bert', 'bert-base-cased',labels=label,args =args, use_cuda=False)

In [ ]:
model = NERModel('bert', 'bert-base-cased',labels=label,args =args, use_cuda=False)

In [ ]:
train2 = train.iloc[:,0:100]
test2 = test.iloc[:,0:100]

In [ ]:
from simpletransformers.classification import ClassificationModel, ClassificationArgs


model_args = ClassificationArgs()
model_args.use_early_stopping = True
model_args.early_stopping_delta = 0.01
model_args.early_stopping_metric = "mcc"
model_args.early_stopping_metric_minimize = False
model_args.early_stopping_patience = 5
model_args.evaluate_during_training_steps = 1000

model = NERModel('bert', 'bert-base-cased',labels=label,args = model_args, use_cuda=False)


In [ ]:
model.train_model(train2,eval_data = test2,acc=accuracy_score)